# 管道模型 (Pipe Model)

`管道 (Pipe)` 类模拟水在管道中的流动。它使用达西-韦斯巴赫方程根据管道两端的压力差（水头）来计算流量。本Jupyter Notebook解释了`Pipe`模型的功能并提供了一个仿真示例。

## 状态变量

管道的状态由以下变量描述：

- `outflow` (float): 计算出的通过管道的流量 (m³/s)。
- `head_loss` (float): 由于管道内摩擦导致的压力水头损失 (m)。

## 参数

管道的物理属性由以下参数定义：

- `length` (float): 管道的长度 (m)。
- `diameter` (float): 管道的直径 (m)。
- `friction_factor` (float): 管道的达西摩擦系数 (无量纲)。

## 仿真示例

以下示例模拟了一个连接两个具有变化水头差的点的管道。仿真演示了出流量如何响应上游水头的变化而变化。结果被绘制成图表以可视化这种关系。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` 和 `numpy` (`pip install matplotlib numpy`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.core.interfaces import State, Parameters

# 管道的初始状态和参数
initial_state = State(outflow=0.0, head_loss=0.0)
parameters = Parameters(
    length=500.0,           # 米
    diameter=0.5,           # 米
    friction_factor=0.02    # 无量纲
)

# 创建一个Pipe实例
pipe = Pipe(name="main_pipe", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 1  # 时间步长（秒）
simulation_duration = 100
num_steps = int(simulation_duration / dt)

# 存储结果用于绘图
history = []

# 定义外部条件
downstream_head = 50.0  # 恒定的下游水头 (m)

# 运行仿真循环
for t in range(num_steps):
    # 为演示目的，随时间使用正弦波改变上游水头
    upstream_head = 55.0 + 5.0 * np.sin(2 * np.pi * t / simulation_duration)
    
    action = {
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = pipe.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", pipe.get_state())

# 准备绘图数据
time_steps = [i * dt for i in range(num_steps)]
outflows = [s['outflow'] for s in history]
head_losses = [s['head_loss'] for s in history]
upstream_heads = [55.0 + 5.0 * np.sin(2 * np.pi * t / simulation_duration) for t in time_steps]

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_steps, outflows, label='出流量 (Outflow)')
ax1.set_ylabel('出流量 (m³/s)')
ax1.set_title('管道流量仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, upstream_heads, label='上游水头 (Upstream Head)', linestyle='--')
ax2.axhline(downstream_head, color='r', linestyle='--', label='下游水头 (Downstream Head)')
ax2.set_xlabel('时间 (秒)')
ax2.set_ylabel('水头 (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
